In [1]:
# Data science in Python Assignment 2| 19200433
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


In [2]:
# function to extracts links from the goven link
def ExtrctLnks4rmCategry(link):
    # requesting data from the link in a package
    WebsiteListPackage=uReq(link)
    #dumping the data in a variable
    WebsiteListHtml=WebsiteListPackage.read()
    # closing the request
    WebsiteListPackage.close()
    # Parsing the html converting into datatype soup
    WebsiteListSoup=soup(WebsiteListHtml,'html.parser')
    # finding all the anchors in the soup and storing in a variable
    WebsiteListId=WebsiteListSoup.findAll('a')
    WebsiteLists=[]
    # loop to extract each link and concatnate it with prerequsite link string
    for a in WebsiteListId:
        WebsiteLists.append('http://mlg.ucd.ie/modules/yalp/'+a['href'])
    WebsiteLists.pop(0)
    return(WebsiteLists)

In [3]:
# function to extract review from each website
def ExtrctReview(WebsiteLists):
    Ratings=[]
    ReviewText=[]
    for i in WebsiteLists:
        WebLink=i
        # requesting data from the link in a package
        WebLinkPackage=uReq(WebLink)
        #dumping the data in a variable
        WebLinkHtml=WebLinkPackage.read()
        # closing the request
        WebLinkPackage.close()
        # Parsing the html converting into datatype soup
        WebReviewSoup=soup(WebLinkHtml,'html.parser')
        # finding all the div's thats have review as their class in the
        #         soup and storing in a variable
        WebReviews=WebReviewSoup.findAll('div',{'class':'review'})
        # loop to store the each rating and text review of a website in lists
        for j in WebReviews:
            Ratings.append(j.find('p',{'class':'rating'}).find('img')['alt'][:1])
#             ReviewText.append(j.text.strip())
            ReviewText.append(j.find('p',{'class':'review-text'}).text.strip())
    return Ratings, ReviewText; 

In [4]:
# function to combine both lists along with a column stating the attitude of review
def convert2Dataframe(Ratings,ReviewText):
    # creating an empty dataframe
    Df = pd.DataFrame()
    # in the empty dataframe creating column for rating and dumping value from the rating list
    Df['Ratings']  = Ratings
    # in the empty dataframe creating column for text review and dumping 
    #     value from the list text review
    Df['ReviewText']  = ReviewText
    # creating a new coulmn based on the condition of positive and negative
    Df['ReviewAttitude'] =['Positive' if (x == '4' or x == '5')  
                           else 'Negative' for x in Df['Ratings']]
    return Df

In [5]:
# for cafe
link='http://mlg.ucd.ie/modules/yalp/cafes_list.html'
# Storing the links as list in variable
cafeLists=ExtrctLnks4rmCategry(link)

In [6]:
# storing the ratings and reviews from each link
CafeRatings, CafeReviewText = ExtrctReview(cafeLists)

In [7]:
# storing both the list as dataframe
cafeDf=convert2Dataframe(CafeRatings,CafeReviewText)

In [12]:
# for hotel
HotelsLink='http://mlg.ucd.ie/modules/yalp/hotels_list.html'
# Storing the links as list in variable
HoteLists=ExtrctLnks4rmCategry(HotelsLink)

In [14]:
# storing the ratings and reviews from each link
HotelRatings, HotelReviewText = ExtrctReview(HoteLists)

In [15]:
# storing both the list as dataframe
HotelDf=convert2Dataframe(HotelRatings,HotelReviewText)

In [16]:
# gym
GymLink='http://mlg.ucd.ie/modules/yalp/gym_list.html'
# Storing the links as list in variable
GymLists=ExtrctLnks4rmCategry(GymLink)

In [17]:
# Storing the links as list in variable
GymRatings, GymReviewText = ExtrctReview(GymLists)

In [18]:
# storing both the list as dataframe
GymDf=convert2Dataframe(GymRatings,GymReviewText)

In [19]:
# function to clean the text
def preProcessing1(text):
    # Make text lowercase
    text = text.lower()
    #remove text in square brackets
    text = re.sub('\[.*?\]', '', text)
    #remove punctuation 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #remove words containing numbers.          
    text = re.sub('\w*\d\w*', '', text)
    #additional punctuation
    text = re.sub('[‘’“”…]', '', text)
    #non-sensical text
    text = re.sub('\n', '', text)
    return text

cleaning = lambda x: preProcessing1(x)

In [20]:
def CleanDataFrame(df):
    # applying cleaning function
    clean = pd.DataFrame(df.ReviewText.apply(cleaning))
    # defining a new coulmn to store ratings
    clean['Ratings']=df.iloc[:,0]
    # defining a new coulmn to store Review Attitude    
    clean['ReviewAttitude']=df.iloc[:,2]
    # storing the order of coulmns in a variable     
    columns=['Ratings','ReviewAttitude','ReviewText']
    # aligning the coulmns as per the redeined order    
    clean=clean.reindex(columns=columns)
    # returning the processed dataframe
    return(clean)


In [21]:
# storing the cleaned data in a variable
cafe_clean=CleanDataFrame(cafeDf)
# Gym_clean

In [22]:
# storing the cleaned data in a variable
Hotel_clean=CleanDataFrame(HotelDf)
# Hotel_clean

In [23]:
# storing the cleaned data in a variable
Gym_clean=CleanDataFrame(GymDf)
# Gym_clean

In [24]:
# function to vectorise dataset
def VectoriseDataFrame(clean):
    # Assigning stopwords to  an object     
    vectorizer = TfidfVectorizer(stop_words='english')
    # transforming the text part of the dataset
    vectors = vectorizer.fit_transform(clean['ReviewText'])
    # getting each word in a variable as the data is transformed now     
    feature_names = vectorizer.get_feature_names()
    # converting the data into a matrix  
    dense = vectors.todense()
    # converting the matrix form to list     
    denselist = dense.tolist()
    # converting newly formed list into dataframe where each word acts like column
    dataset_auto_vectorised = pd.DataFrame(denselist, columns=feature_names)
    return(dataset_auto_vectorised)

In [25]:
# storing vectorised data into a variable
VectorisedCafe=VectoriseDataFrame(cafe_clean)
# VectorisedCafe

In [26]:
# storing vectorised data into a variable
VectorisedHotel=VectoriseDataFrame(Hotel_clean)
# VectorisedHotel

In [27]:
# storing vectorised data into a variable
VectorisedGym=VectoriseDataFrame(Gym_clean)
# VectorisedGym

In [28]:
log=LogisticRegression()

# function to perform logical regression on data 
def LogisticRegression(vectorised,clean):
    # 70% data from dataset is used for training while 30% is used to test the model.     
    X_train, X_test, y_train, y_test = train_test_split(vectorised,
                                                    clean['ReviewAttitude'],
                                                    test_size=0.30)
    # training using Logistic Regression on values where X is the words,Y is their attitude
    log.fit(X_train,y_train)
    # predicting the attitude value for the 30% words
    y4_pred = log.predict(X_test)
    #  storing the accuracy of the prediction in a variable
    acc=accuracy_score(y_test, y4_pred)
    return acc

In [29]:
# accuracy value for cafe
cafe_acc=LogisticRegression(VectorisedCafe,cafe_clean)
cafe_acc

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8

In [30]:
# accuracy value for Hotel
Hotel_acc=LogisticRegression(VectorisedHotel,Hotel_clean)
Hotel_acc

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9

In [31]:
# accuracy value for Gym
Gym_acc=LogisticRegression(VectorisedGym,Gym_clean)
Gym_acc

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.89

In [32]:
# Discussion: all the categories exhibit an accuracy around the range of 80% to 90%.
# the regression associates the presence of certain words with corresponding attitude that 
# was originally in the same row as that of the review text. Acurracy in the range of 80% 
# demonstrates that there are certain words used repetitvely and seem to have the same 
# attitude in repetitive use. there is also the possibility that few words are rarely used 
# and their attitude  may not have been present in the training set but present in test set
# thus are incorrectly classified.

In [33]:
def EvaluatePerformanceOfAOnBnC(CleanA,CleanB,CleanC):
    # Assigning stopwords to  an object
    vectorizer = TfidfVectorizer(stop_words='english')
    # transforming the text part of the dataset
    TfidCleanA= vectorizer.fit(CleanA['ReviewText'])
    # transforming the text of A using object of A
    transCleanA=TfidCleanA.transform(CleanA['ReviewText'])
    # transforming the text of B using object of A
    transCleanB=TfidCleanA.transform(CleanB['ReviewText'])
    # transforming the text of C using object of A
    transCleanC=TfidCleanA.transform(CleanC['ReviewText'])
    
    # getting each word in a variable as the data is transformed now     
    CleanAFeatureNames = TfidCleanA.get_feature_names()
    # converting the data into a matrix  
    CleanADense = transCleanA.todense()
    CleanBDense = transCleanB.todense()
    CleanCDense = transCleanC.todense()
    
    # generating a list item
    CleanADenselist = CleanADense.tolist()
    CleanBDenselist = CleanBDense.tolist()
    CleanCDenselist = CleanCDense.tolist()
    
    # Creating new datasets
    CleanADenselistDf = pd.DataFrame(CleanADenselist, columns=CleanAFeatureNames)
    CleanBDenselistDf = pd.DataFrame(CleanBDenselist, columns=CleanAFeatureNames)
    CleanCDenselistDf = pd.DataFrame(CleanCDenselist, columns=CleanAFeatureNames)
    
    #datasets that is used for training the model     
    Xtrain=CleanADenselistDf
    Y_train=CleanA['ReviewAttitude']
    
    #Training the model on regression     
    log.fit(Xtrain,Y_train)
    
    # data set to be tested
    X_testB=CleanBDenselistDf
    Y_testB=CleanB['ReviewAttitude']
    X_testC=CleanCDenselistDf
    Y_testC=CleanC['ReviewAttitude']
    
    
    # predicting value for each scenario    
    YB_pred = log.predict(X_testB)
    YC_pred = log.predict(X_testC)
    
    B= accuracy_score(Y_testB, YB_pred)
    C= accuracy_score(Y_testC, YC_pred)
    
    return B,C;
    
    

In [34]:
# accuracy score for hotel and gym based on cafe
CafeAccuracy4Hotel,CafeAccuracy4Gym=EvaluatePerformanceOfAOnBnC(cafe_clean,Hotel_clean,Gym_clean)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [35]:
# accuracy score for hotel based on cafe
CafeAccuracy4Hotel

0.7825

In [36]:
# accuracy score for gym based on cafe
CafeAccuracy4Gym

0.774

In [37]:
# accuracy score for cafe and gym based on hotel
HotelAccuracy4Cafe,HotelAccuracy4Gym=EvaluatePerformanceOfAOnBnC(Hotel_clean,cafe_clean,
                                                                 Gym_clean)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [38]:
# accuracy score for cafe based on hotel
HotelAccuracy4Cafe

0.8575

In [39]:
# accuracy score for gym based on hotel
HotelAccuracy4Gym

0.8665

In [40]:
# accuracy score for  Hotel and cafe based on Gym
GymAccuracy4Hotel,GymAccuracy4Cafe=EvaluatePerformanceOfAOnBnC(Gym_clean,Hotel_clean
                                                               ,cafe_clean)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [41]:
# accuracy score for Hotel based on Gym
GymAccuracy4Hotel

0.872

In [42]:
# accuracy score for cafe based on Gym
GymAccuracy4Cafe

0.833